# CNN for Text Classification

In [1]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [ ]:
# The MNIST dataset has 10 classes, representing the digits 0 through 9.
NUM_CLASSES = 10
VOCAB_SIZE = 


In [2]:
# Basic model parameters as external flags.
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_float('momentum', 0.8, 'momentum hyperparameters')
flags.DEFINE_integer('max_steps', 1000, 'Number of steps to run trainer.')
flags.DEFINE_integer('batch_size', 128, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_string('log_dir', 'logs_txt', 'Directory to put the log data.')
flags.DEFINE_float('dropout', 0.5, 'dropout ratio')
flags.DEFINE_float('L2', 0.01, 'Ridge Regularization')
flags.DEFINE_float('L1', 0.01, 'LASSO Regularization')
flags.DEFINE_integer('embedding_size', 10, 'Size of embedding vector')
flags.DEFINE_integer('filter_size', 10, 'Size of embedding vector')
flags.DEFINE_integer('num_filters', 4, 'Number of feature maps')

In [ ]:
def placeholder_inputs(batch_size?, seq_size, num_classes):
    """Generate placeholder variables to represent the input tensors.
    
    Args:
    batch_size: The batch size.?
    seq_size: The sequence size.
    num_classes: The Number of classes
    
    Returns:
    input_x: text sequence placeholder.
    target_y: Labels placeholder.
    """
    input_x = tf.placeholder(tf.float32, [None, seq_size], name="input_x")
    target_y = tf.placeholder(tf.float32, [None, num_classes], name="target_y") # 1-hot-encoded?

    return input_x, num_classes

In [ ]:
def _inference(input_X, vocab_size, embedding_size, ):
    
        
    def weight_variable(shape, name):
        initial = tf.truncated_normal(shape, stddev=1.0 / math.sqrt(float(shape[0])))
        # initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(shape, name):
        #initial = tf.zeros(shape)
        initial = tf.constant(0.1, shape=shape) # prevent dead neural on initialization
        return tf.Variable(initial, name=name)
    
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID') # maintain layer size

    def max_pool(x, seq_size, filter_size):
        return tf.nn.max_pool(x, ksize=[1, sequence_length - filter_size + 1, 1, 1],
                              strides=[1, 1, 1, 1], padding='VALID')
    
    # Embedding layer
    with tf.name_scope('embedding'):
        initial = tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0)
        W = tf.Variable(initial, name="W")
        # perform parallel lookups of embedding
        embedded_chars = tf.nn.embedding_lookup(W, input_X)
        # Add one dimension
        embedded_chars_expan = tf.expand_dims(embedded_chars, dim=-1)
    
    pooled_outputs = []
    for i, filter_size in enumerate(filter_size_list):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            filter_shape = [filter_size, embedding_size, 1, num_filters]

In [ ]:

class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):


        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
